In [ ]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import Aer, execute
# from qiskit.extensions.quantum_initializer.initializer import Initialize
import matplotlib.pyplot as plt
import random
import math

# control is control qubit, target is the list of target qubit
def fanout(qc,control,target):
    length = len(target)
    d = math.ceil(math.log2(length))
    qc_temp = []
    for i in range(d):
        for j in range(length):
            if (2*j+1) <= length-1:
                qc.cx(target[2*j],target[2*j+1])
                qc_temp.append([target[2*j],target[2*j+1]])
            else:
                break
        target = target[::2]
        length = len(target)
    qc.cx(control,target[0])
    for i in range(len(qc_temp)):
        a = qc_temp[len(qc_temp)-1-i][0]
        b = qc_temp[len(qc_temp)-1-i][1]
        qc.cx(a,b)
        
#inl is the list of input qubit, inl[0] dicide the sign of fixed point number, outl is the list of output qubit
def ReLU_core(qc,inl,outl):
    n = len(inl)
    for i in range(n-1):
        qc.h(outl[i])
    for i in range(n-1):
        qc.cx(inl[i+1],outl[i])
    for i in range(n-1):
        qc.tdg(outl[i])
    fanout(qc,inl[0],outl) # first fanout
    for i in range(n-1):
        qc.t(outl[i])
    for i in range(n-1):
        qc.cx(inl[i+1],outl[i])
    for i in range(n-1):
        qc.tdg(outl[i])
    fanout(qc,inl[0],outl) #second fanout
    for i in range(n-1):
        qc.t(outl[i])
        qc.t(inl[i+1])
    temp = (n-1) % 8
    if temp == 1:
        qc.t(inl[0])
    elif temp == 2:
        qc.s(inl[0])
    elif temp == 3:
        qc.t(inl[0])
        qc.s(inl[0])
    elif temp == 4:
        qc.z(inl[0])
    elif temp == 5:
        qc.t(inl[0])
        qc.z(inl[0])
    elif temp == 6:
        qc.s(inl[0])
        qc.z(inl[0])
    elif temp == 7:
        qc.tdg(inl[0])
    #qc.barrier()               #this line can be deleted
    for i in range(n-1):
        qc.h(outl[i])
    fanout(qc,inl[0],inl[1:n]) #third fanout
    for i in range(n-1):
        qc.tdg(inl[i+1])
    fanout(qc,inl[0],inl[1:n]) #last fanout
    
#alpha = 0.125, 0.0625, 0.03125, 0.015625
#code = 0,1  code is 0 means true form, code is 1 means 2's complement
def leaky_ReLU(n,alpha,code,measure,initial):
    shift = 0
    if alpha == 0.125:
        shift = 3
    elif alpha == 0.0625:
        shift = 4
    elif alpha == 0.03125:
        shift = 5
    elif alpha == 0.015625:
        shift == 6
    if shift == 0:
        print("Error: shift is 0")
    qubit_number = n + n + shift
    #the first n qubits represents input qubits, the last n qubits represents ancilla
    qc = QuantumCircuit(qubit_number,qubit_number)
    inl = []
    outl = []
    inl.append(0)
    initial_state = []
    seed = "01"
    initial_state.append(random.choice(seed)[0])
    for i in range(n-1):
        inl.append(i+1)
        outl.append(i+n+1)
        initial_state.append(random.choice(seed)[0])
    for i in range(n+shift):
        initial_state.append('0')
    initial_state.reverse()
    initial_state = ''.join(initial_state)
    print("initial state:",initial_state)
    # if initial == 1:
    #     qc.append(Initialize(initial_state), range(n + n + shift))
    if initial == 1:
        for i in range(n + n + shift):
            if initial_state[n + n + shift - 1 -i] == '1':
                qc.x(i)
    qc.x(0)
    ReLU_core(qc,inl,outl)
    qc.x(0)
    #first part end
    inl = []
    outl = []
    inl.append(0)
    for i in range(n-1):
        inl.append(i+1)
        outl.append(i+n+1+shift)
    ReLU_core(qc,inl,outl)
    qc.cx(0,n)
    #second part end
    if code == 1:
        for i in range(shift):
            qc.cx(0,n+i+1)
    if measure==1:
        qc.barrier()
        for i in range(n + n + shift):
            qc.measure(i,i)
    return qc

In [ ]:
n=15
alpha = 0.0625
code = 1
shift = 0
if alpha == 0.125:
    shift = 3
elif alpha == 0.0625:
    shift = 4
elif alpha == 0.03125:
    shift = 5
elif alpha == 0.015625:
    shift == 6
qc = leaky_ReLU(n,alpha,code,1,1)
simulator = Aer.get_backend('qasm_simulator')
result = execute(qc, simulator, shots=3, memory=True).result()
memory = result.get_memory(qc)
check = 1
for i in range(3):
    inputc = []
    outputc = []
    for j in range(n):
        inputc.append(memory[i][2*n+shift-1-j])
    for j in range(n+shift):
        outputc.append(memory[i][n+shift-1-j])
    print("input[",i,"]",inputc)
    print("output[",i,"]",outputc)